# wikipedia hyperlink network visualization

data: https://snap.stanford.edu/data/wiki-topcats.html

source: Jure Leskovec, Stanford University

> This is a web graph of Wikipedia hyperlinks collected in September 2011. The network was constructed by first taking the largest strongly connected component of Wikipedia, then restricting to pages in the top set of categories (those with at least 100 pages), and finally taking the largest strongly connected component of the restricted graph.
 


## explore the data

the data rests in 3 text files of each ±40MB
- categories
    - a line looks like: Category:Category_name; (series of numbers)
    - Category:Buprestoidea; 301 302 303
    - the numbers presumably represent the pages in the category 
- page names
    - a line looks like: number Page_name
    - 0 Chiasmal syndrome
    - this represents the mappin of number to name
- reduced
    - a line consists of 2 tab separated numbers
    - 52	401135
    - the numbers presumably represent a hyperlink so __edges in the graph__


- use spark 
- load data
- count number of pages ( lines of page doc)
- show all page names in a category 

In [7]:
from pyspark import SparkContext
import pyspark.sql.functions as F

In [14]:
try:
    sc.stop()
    sc = SparkContext('local')
except NameError:
    sc = SparkContext('local')

In [15]:
names = sc.textFile("../data/wiki-topcats-page-names.txt")
edges = sc.textFile("../data/wiki-topcats-reduced.txt")

In [33]:
def getNameByNumber(num):
    num_str = str(num)+' '
    name_filter = names.filter(lambda x: num_str in x[:len(str(num))+1])\
        .map(lambda x: x[len(str(num))+1:])
    name = name_filter.collect()
    return name[0]
    
def getLinksByNumber(num):
    num_str = str(num)+'\t'
    edge_filter = edges.filter(lambda x: num_str in x[:len(str(num))+1])\
        .map(lambda x: x[len(str(num))+1:])
    links = edge_filter.collect()
    return links
def getNextNamesByNumber(num):
    next_nums = getLinksByNumber(num)
    names=[]
    for n in next_nums:
        names.append(getNameByNumber(n))
    return names
def hasLinks(num):
    num_str = str(num)+'\t'
    edge_filter = edges.filter(lambda x: num_str in x[:len(str(num))+1])
    if(edge_filter.count()==0):
        return False
    return True


In [37]:
number = 2
print(getNameByNumber(number))
if(hasLinks(number)):
    print(getNextNamesByNumber(number))
else:
    print("no links")

Pinakion
no links


In [11]:


print(names.take(10))
#lineLengths = lines.map(lambda s: len(s))
#totalLength = lineLengths.reduce(lambda a, b: a + b)

#print(totalLength)

[&#39;0 Chiasmal syndrome&#39;, &#39;1 Kleroterion&#39;, &#39;2 Pinakion&#39;, &#39;3 LyndonHochschildSerre spectral sequence&#39;, &quot;4 Zariski&#39;s main theorem&quot;, &#39;5 FultonHansen connectedness theorem&#39;, &quot;6 Cayley&#39;s ruled cubic surface&quot;, &#39;7 Annulus theorem&#39;, &quot;8 Bing&#39;s recognition theorem&quot;, &#39;9 BochnerMartinelli formula&#39;]
